In [1]:
###获得每一列的名字

name_path='E:/dataset/stockdata/20210101-20220930/name.txt'
name_file = open(name_path,'r',encoding='utf8')
names=name_file.read()
name_file.close()

names=names.split(',')
names=names[0:-1]
colName,remarks=[],[]
for name in names:
    nameLst=name.split('_')
    colName.append(nameLst[1])
    remarks.append(nameLst[0])
colName[0]='R_SecuCode'

print(colName,remarks)

['R_SecuCode', 'CompanyCode', 'Comcd', 'Stkcd', 'Listedstate', 'Csrciccd1', 'Csrciccd2', 'Date', 'PrevClPr', 'Oppr', 'Hipr', 'Lopr', 'Clpr', 'AdjClpr1', 'AdjClpr2', 'Trdvol', 'Trdsum', 'Dampltd', 'DFulTurnR', 'DTrdTurnR', 'Capchgdt', 'Comstateshr', 'Comlpshr', 'Fullshr', 'Trdshr', 'Lsttrdshr', 'Dret', 'Daret', 'PE', 'PB', 'PCF', 'PS', 'EPS', 'ROE', 'AccumFundPS', 'OpPrfPS', 'NAPS', 'NAPSadj', 'IncomePS', 'NCFfropePS'] ['证券代码', '公司代码', '上市公司代码', '股票代码', '上市状态', '证监会行业门类代码', '证监会行业大类代码', '日期', '前收盘价', '开盘价', '最高价', '最低价', '收盘价', '复权价1(元)', '复权价2(元)', '成交量', '成交金额', '日振幅(%)', '总股数日换手率(%)', '流通股日换手率(%)', '股数变动日', '公司国有股', '公司法人股', '总股数', '流通股', '已上市流通股', '日收益率', '日资本收益率', '市盈率', '市净率', '市现率', '市销率', '每股收益(摊薄)(元/股)', '净资产收益率(摊薄)', '每股公积金(元/股)', '每股营业利润(元/股)', '每股净资产(元/股)', '调整后每股净资产(元/股)', '每股营业收入', '每股经营活动现金流量净额(元/股)']


In [2]:
#####获得每一句insert 语句

sqls_insert_sdata=[]

for i in range(0,19):
    data_path='E:/dataset/stockdata/20210101-20220930/RESSET_DRESSTK_2021__'
    skipFirst = True
    data_path = data_path+ str(i+1)+".csv"
    for line in open(data_path,'r',encoding='utf8'):
        if skipFirst:
            skipFirst = False
            continue
        data = line.replace('"','').replace('\n','').split(',')
        sql_insert_sdata = "insert into comprehensive_Sdata value("
        sql_insert_sdata =sql_insert_sdata+str(data).replace('[','').replace(']','').replace('\'\'','NULL')+");"
        sqls_insert_sdata.append(sql_insert_sdata)


In [3]:
len(sqls_insert_sdata)

1854550

In [4]:
import pymysql
import time
import logging
import os

def get_debug_logger(file_log,name):
    if not os.path.exists(file_log):
        os.makedirs(file_log)
    # 获取logger对象,取名mylog
    logger = logging.getLogger(name)
    # 输出DEBUG及以上级别的信息，针对所有输出的第一层过滤
    logger.setLevel(logging.DEBUG)
    # 获取文件日志句柄并设置日志级别，第二层过滤
    handler = logging.FileHandler(file_log)
    handler.setLevel(logging.DEBUG)
    # 生成并设置文件日志格式，其中name为上面设置的mylog
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    handler.setFormatter(formatter)
    logger.addHandler(handler)
    return logger
def get_error_logger(file_log,name):
    if not os.path.exists(file_log):
        os.makedirs(file_log)
    # 获取logger对象,取名mylog
    logger = logging.getLogger(name)
    # 输出DEBUG及以上级别的信息，针对所有输出的第一层过滤
    logger.setLevel(logging.ERROR)
    # 获取文件日志句柄并设置日志级别，第二层过滤
    handler = logging.FileHandler(file_log)
    handler.setLevel(logging.ERROR)
    # 生成并设置文件日志格式，其中name为上面设置的mylog
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    handler.setFormatter(formatter)
    logger.addHandler(handler)
    return logger
def get_logger():
    date=time.localtime(time.time())
    num_logs=0
    logs_path = './log/logs'
    logs_path=logs_path+"/log_%d_%d_%d_%d"%(date[0],date[1],date[2],num_logs)+".txt"
    # num_errors=0
    # errors_path = './log/errors'
    # errors_path=errors_path+"/error_%d_%d_%d_%d"%(date[0],date[1],date[2],num_errors)+".txt"

    logger = get_debug_logger(logs_path,"logs")
    return logger

In [11]:
#####数据导入数据库
db = pymysql.connect(host='localhost',
                     user='qauser',
                     password='123456',
                     database='qadata',
                     charset='utf8')
# logger = get_logger()



cur = db.cursor()
start = time.time()
for sql in sqls_insert_sdata:
    # try :
    cur.execute(sql)
    db.commit()
        # logger.info("SUCESS")
    # except Exception as err:
        # logger.error("data:"+sql+" REASON:"+err)




In [14]:
import pymysql
import time
import logging
import os
db = pymysql.connect(host='localhost',
                     user='qauser',
                     password='123456',
                     database='qadata',
                     charset='utf8')
cur = db.cursor()
sql = "select * from (select Stkcd,count(*) as num,max(Date) as flag from comprehensive_Sdata group by Stkcd )as a where flag='2022-09-30' and num >1;"
sql = "select count(*) from comprehensive_Sdata;"
cur.execute(sql)
result = cur.fetchall()
print(result)

((1854550,),)
